In [1]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
import os
import logging
import configparser

In [2]:
spark = SparkSession.builder \
    .appName("appGlob") \
    .config("spark.jars","../controller/postgresql-42.7.2.jar")\
    .config('spark.driver.extraClassPath', '../controller/postgresql-42.7.2.jar') \
    .getOrCreate()

In [3]:
#log
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("ReadCSV")


In [4]:
config = configparser.ConfigParser()
config.read("../properties.conf")
database_config = config["database"]


In [5]:
#LOAD_FILE
filePath = "C:/Users/andre/OneDrive/Escritorio/data_challenge_files/"
fileType = "jobs.csv"
# Define the schema
dp_schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("job", StringType(), True)
])


In [6]:
try: 
    df = spark.read.csv(filePath+fileType,header=False, inferSchema=False, schema=dp_schema ,sep=",")
    
except Exception as e:
    logger.error("Error: %s", str(e), exc_info=True)
    df = spark.createDataFrame([], schema=schema)

In [7]:
logger.info("Dataframe schema for jobs tale :")
df.printSchema()

INFO:ReadCSV:Dataframe schema for jobs tale :


root
 |-- id: integer (nullable = true)
 |-- job: string (nullable = true)



In [8]:
df.count()

183

In [9]:
try :
    df.write \
    .format("jdbc") \
    .mode("overwrite") \
    .option("url", database_config["url"]) \
    .option("dbtable", "jobs") \
    .option("user", database_config["user"]) \
    .option("password", database_config["password"]) \
    .option("driver", database_config["driver"]) \
    .save()
except Exception as e:
    logger.error("Erro: review eschemas or datatypes %s", str(e), exc_info=True)